# Cashflow management problem

Details of the problem are as follows:
+ Liability = [100.0,500.0,100.0,-600.0,-500.0, 200.0, 600.0, -900.0, 0.0]
+ Financial Instruments
    + Quarterly loan: 2.5% per month
    + Two-quarter loan: 1.8% for two months
    + Two-year loan: 1.0% compounded for eight months (can only take on first month of series-->Q1)
    + Risk-free Rate (Cash): Unlimited, interest = 0.5% per month

+ Goal: Maximize the end of period wealth in quarter number nine


Import the relevant python modules

In [ ]:
import numpy as np #import numpy for matrix and array usage
from tabulate import *
import cvxpy as cvx # cvxpy package allows us to optimize

Set up the data for the problem

In [ ]:
#array for liabilities and cash flows
liability = np.array([100.0,500.0,100.0,-600.0,-500.0, 200.0, 600.0, -900.0, 0.0])
# of periods in series
ntimes = len(liability);
#periods for two-year and two-quarter loans
two_yr = 8;
two_qtr = 2;
#interest rates for 2 year loan, 2 quarter loan, quarterly loan, and risk-free rate
two_yr_int = 1.0;
two_qtr_int = 1.8;
qtr_int = 2.5;
rf_int = 0.5;

Set up the variables

In [ ]:
#variable representing Two-year loan borrowing
w = cvx.Variable(ntimes-two_yr)
#variable representing how much we borrow from Quarterly loan
x = cvx.Variable(ntimes-1)
#variable representing Two-quarter loan borrowing
y = cvx.Variable(ntimes-two_qtr)
#variable showing excess cash surplus/deficit each period
z = cvx.Variable(ntimes)

Set up the constraints for each time period

In [ ]:
constraints = [];
# flow constraints for each time period
for t in range(ntimes):
    expr = []
    #start at time 1 (no 0) to start counting Quarterly loan
    if (t>0):
        expr += [-(1+qtr_int/100)*x[t-1]];
    if (t<ntimes-1):
        expr += [x[t]]
    #starts at time 3 (no 0) for Two-quarter loan given timeframe
    if (t>two_qtr-1):
        expr += [-((1+two_qtr_int/100)**2)*y[t-two_qtr]];
    if (t<ntimes-two_qtr):
        expr += [y[t]];
    #starts at time 8 (no 0) for Two-year loan given timeframe
    if (t>two_yr-1):
        expr += [-((1+two_yr_int/100)**8)*w[t-two_yr]];
    if (t<ntimes-two_yr):
        expr += [w[t]];
    #start at time 1 (not 0) for cash investment returns
    if (t>0):
        expr += [(1+rf_int/100)*z[t-1]]; #interest on investments (rfr)
    expr += [-z[t]];
    constraints += [sum(expr) >= liability[t]];

# non-negativity constraints
constraints += [x>=0, y>=0, w>=0, z>=0];

# objective function
objective = cvx.Maximize(z[ntimes-1]);

Set up the optimization and solve it using the default cvxpy solver

In [ ]:
#Solves optimization problem and prints the answer and variable values below
prob = cvx.Problem(objective, constraints)
prob.solve()

np.set_printoptions(precision=3,suppress=True)
print('Problem status: ' + str(prob.status));
if (prob.status == 'optimal'):
    print('Variable values (Loans & Cash): %.2f' % prob.value);
    print('Two-year loan value: ')
    print(w.value.transpose())
    print('Two-quarter loan value: ')
    print(y.value.transpose())
    print('Quarter loan value: ')
    print(x.value.transpose())
    print('Cash value: ')
    print(z.value.transpose())

Problem status: optimal
Variable values (Loans & Cash): 471.56
Two-year loan value: 
[399.81]
Two-quarter loan value: 
[  0.    198.691   0.      0.      0.      0.      0.   ]
Quarter loan value: 
[  0.   0. 100.   0.   0.   0.   0.   0.]
Cash value: 
[299.81    0.      0.    291.592 793.05  597.015   0.    900.    471.563]


Solve the same LP using GLPK_MI

In [ ]:
#Solves optimization problem and prints the answer and variable values below
print(cvx.installed_solvers())
prob.solve(solver=cvx.GLPK_MI)

np.set_printoptions(precision=2,suppress=True)
print('Problem status: ' + str(prob.status));
if (prob.status == 'optimal'):
    print('Variable values (Loans & Cash): %.2f' % prob.value);
    print('Two-year loan value: ')
    print(w.value.transpose())
    print('Two-quarter loan value: ')
    print(y.value.transpose())
    print('Quarter loan value: ')
    print(x.value.transpose())
    print('Cash value: ')
    print(z.value.transpose())

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']
Problem status: optimal
Variable values (Loans & Cash): 471.56
Two-year loan value: 
[399.81]
Two-quarter loan value: 
[ -0.   198.69  -0.    -0.    -0.    -0.    -0.  ]
Quarter loan value: 
[ -0.  -0. 100.  -0.  -0.  -0.  -0.  -0.]
Cash value: 
[299.81  -0.    -0.   291.59 793.05 597.01  -0.   900.   471.56]
